
# Ceph Manager — Operator Notebook

Use this notebook to **add/delete CephX users** and **add/resize/delete CephFS subvolumes**, either **one at a time** or **in bulk from CSV files**.

This notebook expects the Python client `CephManagerClient` to be importable (from your package or as a module).


In [1]:
from typing import Any, Dict, List, Optional
from dataclasses import dataclass
import json
import pandas as pd
from fabric_ceph_client.fabric_ceph_client import CephManagerClient

In [6]:

BASE_URL = "https://23.134.232.53"   # e.g., "https://mgr.example.org/api"
TOKEN = "/Users/kthare10/work/id_token_prod.json"                           # paste a JWT if your service requires it, else None
VERIFY_TLS = False                      # set False only for testing self-signed certs

client = CephManagerClient(base_url=BASE_URL, token_file=TOKEN, verify=VERIFY_TLS)

TypeError: CephManagerClient.__init__() got an unexpected keyword argument 'token_file'


## CephX user helpers
Single operations and CSV batch upload.


In [3]:

def upsert_user(user_entity: str, capabilities: List[Dict[str, str]], x_cluster: Optional[str]=None) -> Dict[str, Any]:
    """Create or update a CephX user. Server may implement upsert; this calls POST first, then falls back to PUT."""
    try:
        try:
            return client.create_user(user_entity, capabilities, x_cluster=x_cluster)
        except ApiError as e:
            # if user exists or POST is restricted, try PUT
            return client.update_user(user_entity, capabilities, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during upsert_user:", e)
        raise

def delete_user(entity: str, x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.delete_user(entity, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during delete_user:", e)
        raise

def export_keyrings(entities: List[str], x_cluster: Optional[str]=None) -> str:
    try:
        return client.export_users(entities, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during export_keyrings:", e)
        raise

def list_users() -> str:
    try:
        return client.list_users()
    except ApiError as e:
        print("API error during list_users:", e)
        raise


## CephFS subvolume helpers
Create/resize/delete subvolumes and check/get info. `mode` is used on **create**; `size` sets quota in bytes.


In [4]:

def ensure_group(vol_name: str, group_name: str, x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.create_subvolume_group(vol_name, group_name, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during ensure_group:", e)
        raise

def create_or_resize_subvol(vol_name: str, subvol_name: str, *, group_name: Optional[str]=None,
                            size_bytes: Optional[int]=None, mode: Optional[str]=None,
                            x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.create_or_resize_subvolume(vol_name, subvol_name, group_name=group_name,
                                                 size=size_bytes, mode=mode, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during create_or_resize_subvol:", e)
        raise

def get_subvol_info(vol_name: str, subvol_name: str, *, group_name: Optional[str]=None,
                    x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.get_subvolume_info(vol_name, subvol_name, group_name=group_name, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during get_subvol_info:", e)
        raise

def subvol_exists(vol_name: str, subvol_name: str, *, group_name: Optional[str]=None,
                  x_cluster: Optional[str]=None) -> bool:
    try:
        return client.subvolume_exists(vol_name, subvol_name, group_name=group_name, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during subvol_exists:", e)
        raise

def delete_subvol(vol_name: str, subvol_name: str, *, group_name: Optional[str]=None, force: bool=False,
                  x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.delete_subvolume(vol_name, subvol_name, group_name=group_name, force=force, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during delete_subvol:", e)
        raise



## One-at-a-time examples
Uncomment and run what you need.


In [5]:
print(list_users())

/Users/kthare10/renci/code/fabric/1.6/fabric_ceph/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '23.134.232.53'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/kthare10/renci/code/fabric/1.6/fabric_ceph/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '23.134.232.53'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/kthare10/renci/code/fabric/1.6/fabric_ceph/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '23.134.232.53'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.i

NameError: name 'ApiError' is not defined

In [ ]:

# ---- CephX user: upsert
# caps = [
#     {"entity": "mon", "cap": "allow r"},
#     {"entity": "mds", "cap": "allow rw fsname=CEPH-FS-01 path=/volumes/_nogroup/demo"},
#     {"entity": "osd", "cap": "allow rw tag cephfs data=CEPH-FS-01"},
#     {"entity": "osd", "cap": "allow rw tag cephfs metadata=CEPH-FS-01"},
# ]
# upsert_user("client.demo", caps)

# ---- CephX user: export keyring(s)
# print(export_keyrings(["client.demo"]))

# ---- CephX user: delete
# delete_user("client.demo")

# ---- CephFS: ensure group, create, info, delete
# ensure_group("CEPH-FS-01", "fabric_staff")
# create_or_resize_subvol("CEPH-FS-01", "alice", group_name="fabric_staff", size_bytes=10 * 1024**3, mode="0777")
# info = get_subvol_info("CEPH-FS-01", "alice", group_name="fabric_staff"); info
# delete_subvol("CEPH-FS-01", "alice", group_name="fabric_staff")



## Batch from CSV — CephX users

**CSV format (`users.csv`)**

| user_entity | capabilities_json |
|-------------|-------------------|
| client.demo | [{ "entity":"mon","cap":"allow r" }, { "entity":"mds","cap":"allow rw" }] |

- `capabilities_json` is a JSON array of objects with `entity` and `cap` keys.


In [ ]:

from caas_jupyter_tools import display_dataframe_to_user

def process_users_csv(csv_path: str, *, action: str="upsert", x_cluster: Optional[str]=None) -> pd.DataFrame:
    """action: 'upsert' or 'delete'"""
    df = pd.read_csv(csv_path)
    results = []
    for _, row in df.iterrows():
        entity = str(row["user_entity"]).strip()
        try:
            if action == "upsert":
                caps = json.loads(row["capabilities_json"])
                _ = upsert_user(entity, caps, x_cluster=x_cluster)
                results.append({"user_entity": entity, "action": "upsert", "status": "ok"})
            elif action == "delete":
                _ = delete_user(entity, x_cluster=x_cluster)
                results.append({"user_entity": entity, "action": "delete", "status": "ok"})
            else:
                results.append({"user_entity": entity, "action": action, "status": "unsupported"})
        except ApiError as e:
            results.append({"user_entity": entity, "action": action, "status": f"error: {e.message}"})
    out = pd.DataFrame(results)
    display_dataframe_to_user("users_batch_results", out)
    return out

# Example usage:
# process_users_csv("/path/to/users.csv", action="upsert")
# process_users_csv("/path/to/users.csv", action="delete")



## Batch from CSV — CephFS subvolumes

**CSV format (`subvols.csv`)**

| vol_name   | subvol_name | group_name   | action  | size_bytes | mode  |
|------------|-------------|--------------|---------|------------|-------|
| CEPH-FS-01 | alice       | fabric_staff | create  | 10737418240| 0777  |
| CEPH-FS-01 | alice       | fabric_staff | resize  | 536870912  |       |
| CEPH-FS-01 | alice       | fabric_staff | delete  |            |       |

- `action` is one of `create`, `resize`, `delete`
- `size_bytes` is optional for `create` (omit → unlimited); required for `resize`
- `mode` is only used on create


In [ ]:

def process_subvols_csv(csv_path: str, x_cluster: Optional[str]=None) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    results = []
    for _, row in df.iterrows():
        fs = str(row["vol_name"]).strip()
        name = str(row["subvol_name"]).strip()
        group = None if pd.isna(row.get("group_name")) else str(row.get("group_name")).strip()
        action = str(row["action"]).strip().lower()
        size = row.get("size_bytes")
        size = int(size) if (pd.notna(size)) else None
        mode = row.get("mode")
        mode = None if pd.isna(mode) or str(mode).strip()=="" else str(mode).strip()

        try:
            if action in ("create", "resize"):
                if group:
                    ensure_group(fs, group, x_cluster=x_cluster)
                _ = create_or_resize_subvol(fs, name, group_name=group, size_bytes=size, mode=mode, x_cluster=x_cluster)
                info = get_subvol_info(fs, name, group_name=group, x_cluster=x_cluster)
                path = info.get("path", "")
                results.append({"vol_name": fs, "subvol_name": name, "action": action, "status": "ok", "path": path})
            elif action == "delete":
                _ = delete_subvol(fs, name, group_name=group, x_cluster=x_cluster)
                results.append({"vol_name": fs, "subvol_name": name, "action": action, "status": "ok"})
            else:
                results.append({"vol_name": fs, "subvol_name": name, "action": action, "status": "unsupported"})
        except ApiError as e:
            results.append({"vol_name": fs, "subvol_name": name, "action": action, "status": f"error: {e.message}"})
    out = pd.DataFrame(results)
    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user("subvols_batch_results", out)
    return out

# Example usage:
# process_subvols_csv("/path/to/subvols.csv")
